In [25]:
import gym
import numpy as np

env = gym.make("Taxi-v3").env

print("Action Space {}".format(env.action_space))
print("State Space {}".format(env.observation_space))

state = env.encode(3, 1, 2, 0)  # (taxi row, taxi column, passenger index, destination index)
print("State:", state)
env.s = state
# env.render()

env.P[328]

Action Space Discrete(6)
State Space Discrete(500)
State: 328


{0: [(1.0, 428, -1, False)],
 1: [(1.0, 228, -1, False)],
 2: [(1.0, 348, -1, False)],
 3: [(1.0, 328, -1, False)],
 4: [(1.0, 328, -10, False)],
 5: [(1.0, 328, -10, False)]}

In [26]:

epochs = 0
penalties, reward = 0, 0

done = False

while not done:
    action = env.action_space.sample()
    state, reward, done, info = env.step(action)

    if reward == -10:
        penalties += 1

    epochs += 1

print("Timesteps taken: {}".format(epochs))
print("Penalties incurred: {}".format(penalties))

Timesteps taken: 705
Penalties incurred: 240


In [27]:
q_table = np.zeros([env.observation_space.n, env.action_space.n])
print(q_table)
print(q_table.shape)

[[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 ...
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]
(500, 6)


In [28]:
import random
from IPython.display import clear_output

# Hyperparameters
alpha = 0.1
gamma = 0.6
epsilon = 0.1

# For plotting metrics
all_epochs = []
all_penalties = []

for i in range(1, 10000):
    state = env.reset()

    epochs, penalties, reward, = 0, 0, 0
    done = False

    while not done:
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample() # Explore action space
        else:
            action = np.argmax(q_table[state]) # Exploit learned values

        next_state, reward, done, info = env.step(action)

        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state])

        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        q_table[state, action] = new_value

        if reward == -10:
            penalties += 1
            # print('penalizacion!!')

        state = next_state
        epochs += 1

    if i % 100 == 0:
        clear_output(wait=True)
        print(f"Episode: {i}")

print("Training finished.\n")

Episode: 9900
Training finished.



In [29]:
# q_table[328]
print(q_table)
print(q_table.shape)

[[ 0.          0.          0.          0.          0.          0.        ]
 [-2.38906236 -2.34135906 -2.366458   -2.34588691 -2.27325184 -9.72097211]
 [-1.83194116 -1.49003483 -1.85143585 -1.56230359 -0.7504     -8.90978728]
 ...
 [-1.26757871 -0.10979343 -1.28944214 -1.28942249 -2.82526002 -3.60072843]
 [-2.08456502 -2.07872663 -2.0828624  -2.07565732 -5.05711713 -3.65815118]
 [-0.196      -0.196       0.35710964 10.96830438 -1.906      -1.80241258]]
(500, 6)


In [30]:
"""Evaluate agent's performance after Q-learning"""

total_epochs, total_penalties, total_aciertos = 0, 0, 0
episodes = 100

for _ in range(episodes):
    state = env.reset()
    epochs, penalties, reward = 0, 0, 0
    acierto=0

    done = False

    # print('cuantas',_)

    while epochs<150 and reward != -10 and not done:
        action = np.argmax(q_table[state])
        state, reward, done, info = env.step(action)
        # print('bucle while')

        if reward == -10 or epochs == 149:
            penalties += 1
            # print('penalizacion!')

        if done:
            # print('acierto')
            acierto += 1
        epochs += 1


    total_penalties += penalties
    total_epochs += epochs
    total_aciertos += acierto

print(f"Results after {episodes} episodes:")
print(f"Total penalties: {total_penalties}")
print(f"Total aciertos: {total_aciertos}")
# print(f"Average timesteps per episode: {total_epochs / episodes}")
# print(f"Average penalties per episode: {total_penalties / episodes}")


Results after 100 episodes:
Total penalties: 11
Total aciertos: 89
